In [5]:
from selenium import webdriver
import pymssql
from selenium.webdriver.common.action_chains import ActionChains
import time
import os
import pandas as pd
import warnings
warnings.filterwarnings("ignore")  # 忽略UserWarning兼容性警告
date = time.strftime("%Y-%m-%d")

In [6]:
output_folder = f'C:/Users/15516/Desktop/数据处理/励齿数据/{date}'

if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    print(f"文件夹 {output_folder} 创建成功")
else:
    print(f"文件夹 {output_folder} 已存在")

文件夹 C:/Users/15516/Desktop/数据处理/励齿数据/2023-07-06 已存在


# 获取源文件
## 获取商品后台数据

### 配置函数

In [7]:
def Download(url):
    driver = webdriver.Chrome()  # 启动谷歌浏览器
    url = url  # 获取上海后台网址
    driver.get(url)  # 登录网址
    time.sleep(3)  # 等待网页加载五秒

    # 输入账号和密码
    driver.find_element('xpath','//input[@type="text"]').send_keys('韦鹏')
    driver.find_element('xpath','//input[@type="password"]').send_keys('wp123456')

    time.sleep(1)

    # 点击登录按钮
    driver.find_element('xpath','//button').click()
    time.sleep(1)

    # 点击按钮1
    # from selenium.webdriver.common.by import By

    # button1 = driver.find_element(By.XPATH, './/li[contains(text(),"子产品管理")]')

    button_first = driver.find_element('xpath', "//span[contains(text(),'产品列表')]")
    button_first.click()
    time.sleep(1)

    # 点击按钮2
    button_second = driver.find_element('xpath',"//li[contains(text(),'子产品管理')]")
    ActionChains(driver).click(button_second).perform()
    time.sleep(1)

    # 点击按钮2
    button_download = driver.find_element('xpath',"//span[contains(text(),'导出商品详情')]")
    button_download.click()
    time.sleep(50)
    
    if os.path.exists('C:/Users/15516/Downloads/子产品详情.xlsx'):
        print("文件存在，下载完成")
    else:
        print("文件不存在")
    
    driver.quit() # 关闭谷歌浏览器

    
def Deal_with(output_folder,area):
    # 读取文件并保存到新路径
    file = 'C:/Users/15516/Downloads/子产品详情.xlsx'
    file_df = pd.read_excel(file, sheet_name='商品详情')
    file_df.to_excel(f'{output_folder}/{area}产品详情.xlsx', index=False)
    print('文件创建成功')

    # 删除原文件
    os.remove(file)

In [8]:
Download('https://www.lichidental.com/sys/index.html#/')
Deal_with(output_folder,"上海")

文件存在，下载完成
文件创建成功


In [9]:
Download('https://www.beijinglichidental.com/sys/#/')
Deal_with(output_folder,"北京")

文件存在，下载完成
文件创建成功


In [10]:
Download('https://www.lichigz.com/sys/#/')
Deal_with(output_folder,"广州")

文件存在，下载完成
文件创建成功


# 数据库数据提取

In [11]:
server = "120.253.186.206:12513"
user = "baseline2"
password = "qwert123!@#"
database = "Lichi"  

# 建立数据库连接
conn = pymssql.connect(server=server, user=user, password=password, database=database)

In [12]:
# 采购价格获取
query_sh_Purchase_Price = '''
SELECT
    de.cinvcode AS '励齿编码',
    de.cInvName AS '商品名称',
    SUM(de.iGquantity) AS '数量',
    SUM(de.iGquantity * de.iUnitPrice) / SUM(de.iGquantity) AS '不含税采购价',
    SUM(de.iGquantity * de.iTaxUnitPrice) / SUM(de.iGquantity) AS '含税采购价'
FROM
    lichi..PU_PurOrderDetail de
    INNER JOIN lichi..PU_PurOrderMaster ma 
    ON de.POMID = ma.POMID
WHERE
    DATEDIFF(dd, ma.dInputDate, GETDATE()) <= 30
    AND ma.Status in ('已审核','关闭')
    AND de.iTaxUnitPrice NOT IN ('0', '0.01', '0.1')
    AND (SELECT cVenName from lichi..vendor where ma.cVenCode = cVenCode) not like '%励齿医疗%' 
    AND igQuantity != 0
GROUP BY
    de.cinvcode,
    de.cInvName
'''

query_bj_Purchase_Price = '''
SELECT
    de.cinvcode AS '励齿编码',
    de.cInvName AS '商品名称',
    SUM(de.iGquantity) AS '数量',
    SUM(de.iGquantity * de.iUnitPrice) / SUM(de.iGquantity) AS '不含税采购价',
    SUM(de.iGquantity * de.iTaxUnitPrice) / SUM(de.iGquantity) AS '含税采购价'
FROM
    LiChi_BeiJing..PU_PurOrderDetail de
    INNER JOIN LiChi_BeiJing..PU_PurOrderMaster ma 
    ON de.POMID = ma.POMID
WHERE
    DATEDIFF(dd, ma.dInputDate, GETDATE()) <= 30
    AND ma.Status in ('已审核','关闭')
    AND de.iTaxUnitPrice NOT IN ('0', '0.01', '0.1')
    AND (SELECT cVenName from LiChi_BeiJing..vendor where ma.cVenCode = cVenCode) not like '%励齿医疗%'
    AND igQuantity != 0
GROUP BY
    de.cinvcode,
    de.cInvName
'''

query_gz_Purchase_Price = '''
SELECT
    de.cinvcode AS '励齿编码',
    de.cInvName AS '商品名称',
    SUM(de.iGquantity) AS '数量',
    SUM(de.iGquantity * de.iUnitPrice) / SUM(de.iGquantity) AS '不含税采购价',
    SUM(de.iGquantity * de.iTaxUnitPrice) / SUM(de.iGquantity) AS '含税采购价'
FROM
    LiChi_GuangZhou..PU_PurOrderDetail de
    INNER JOIN LiChi_GuangZhou..PU_PurOrderMaster ma 
    ON de.POMID = ma.POMID
WHERE
    DATEDIFF(dd, ma.dInputDate, GETDATE()) <= 30
    AND ma.Status in ('已审核','关闭')
    AND de.iTaxUnitPrice NOT IN ('0', '0.01', '0.1')
    AND (SELECT cVenName from LiChi_GuangZhou..vendor where ma.cVenCode = cVenCode) not like '%励齿医疗%'
    AND igQuantity != 0
GROUP BY
    de.cinvcode,
    de.cInvName
'''

In [13]:
def lichi_Purchase_Price(query,area):
    """
    根据给定的SQL查询，从数据库中获取采购价格数据，并将结果保存为Excel文件。

    参数：
    - query: SQL查询语句
    - area: 区域名称，用于生成输出文件的文件名

    返回：
    无

    """
    pd.read_sql(query,conn).to_excel(f'{output_folder}/{area}采购价格.xlsx',sheet_name='采购价格',index = False)

In [14]:
lichi_Purchase_Price(query_sh_Purchase_Price,"上海")
lichi_Purchase_Price(query_bj_Purchase_Price,"北京")
lichi_Purchase_Price(query_gz_Purchase_Price,"广州")

In [15]:
# 库存获取
query_sh_Inventory = '''
SELECT
    cInvCode '励齿编码',
    SUM ( iQuantity ) '励齿库存' 
FROM
    LiChi.dbo.WMS_CurrentStock 
WHERE
    cWhCode = '01' 
GROUP BY
    cInvCode
'''

query_bj_Inventory = '''
SELECT
    cInvCode '励齿编码',
    SUM ( iQuantity ) '励齿库存' 
FROM
    LiChi_BeiJing..WMS_CurrentStock 
WHERE
    cWhCode = '001' 
GROUP BY
    cInvCode
'''

query_gz_Inventory = '''
SELECT
    cInvCode '励齿编码',
    SUM (iQuantity) '励齿库存' 
FROM
    LiChi_GuangZhou..WMS_CurrentStock 
WHERE
    cWhCode = '88' 
GROUP BY
    cInvCode
'''

In [16]:
def lichi_Inventory(query,area):
    """
    根据给定的SQL查询，从数据库中获取采购价格数据，并将结果保存为Excel文件。

    参数：
    - query: SQL查询语句
    - area: 区域名称，用于生成输出文件的文件名

    返回：
    无

    """
    pd.read_sql(query,conn).to_excel(f'{output_folder}/{area}库存.xlsx',sheet_name='库存',index = False)

In [17]:
lichi_Inventory(query_sh_Inventory,"上海")
lichi_Inventory(query_bj_Inventory,"北京")
lichi_Inventory(query_gz_Inventory,"广州")